In [40]:
from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [23]:
# selecting model checkpoint
model_checkpoint = "t5-small"

In [51]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the WMT16 dataset
data = pd.read_csv('../data/interim/training_data.csv', index_col=False)

# Load the BLUE metric
metric = load_metric("sacrebleu")
print(data.isna().sum())

reference      0
translation    0
dtype: int64


In [52]:
from transformers import AutoTokenizer

# we will use autotokenizer for this purpose
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [53]:
prefix = "Make this text less toxic:"

In [54]:
import pyarrow as pa
from datasets import Dataset, DatasetDict

dataset = Dataset(pa.Table.from_pandas(data.reset_index(drop=True)))

In [55]:
train_test_split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Split the train set further into training and validation sets
train_val_split = train_test_split_dataset['train'].train_test_split(test_size=0.1, seed=42)

# Combine the splits into a single DatasetDict
final_splits = DatasetDict({
    'train': train_val_split['train'],
    'validation': train_val_split['test'],
    'test': train_test_split_dataset['test']
})

final_splits["train"][:2]

{'reference': ['your parents must have fucked you up',
  'shes the fastest hunk of junk in the galaxy'],
 'translation': ['your mom and dad mustve spoiled you a lot',
  'its the fastest can in the galaxy']}

In [56]:
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [57]:
tokenized_datasets = final_splits.map(preprocess_function, batched=True)
tokenized_datasets['train'][0]

Map:   0%|          | 0/427669 [00:00<?, ? examples/s]

Map:   0%|          | 0/47519 [00:00<?, ? examples/s]

Map:   0%|          | 0/52799 [00:00<?, ? examples/s]

{'reference': 'your parents must have fucked you up',
 'translation': 'your mom and dad mustve spoiled you a lot',
 'input_ids': [1796,
  48,
  1499,
  705,
  12068,
  10,
  13055,
  1362,
  398,
  43,
  3,
  89,
  15318,
  25,
  95,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [39, 3912, 11, 7370, 398, 162, 3, 29793, 25, 3, 9, 418, 1]}

In [62]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [63]:
# defining the parameters for training
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"text-detoxifier-t5-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    # fp16=True,
)

In [64]:
# instead of writing collate_fn function we will use DataCollatorForSeq2Seq
# simliarly it implements the batch creation for training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [65]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [66]:
# instead of writing train loop we will use Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [67]:
trainer.train()

  0%|          | 0/40095 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 